In [1]:
import requests
import json
import pandas as pd

headers = { 'Content-Type': 'application/json' }
url = 'https://us-central1-magnetic-flare-397719.cloudfunctions.net/model-server'

In [2]:
test_df = pd.read_csv('../../data/transactions-test.csv')
test_df.head(3)

,Class,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,0,0.784347,2.094837,-1.059579,-0.242263,-0.839254,-1.313586,-0.491879,-1.188949,-0.006994,...,0.018661,0.528221,1.614207,0.056421,0.148257,-0.123297,0.011339,0.017207,-0.054294,-0.282063
1,0,-0.993889,-0.361575,1.109193,1.187961,-0.135402,0.291302,-0.457649,0.640618,0.064412,...,0.170866,-0.227156,-0.593673,-0.072721,-0.033578,-0.128726,0.072962,0.244652,0.085972,-0.337757
2,0,1.523733,2.117582,0.347923,-2.353891,0.775983,1.428246,-0.160250,0.676584,-0.259292,...,-0.184910,0.103705,0.380105,-0.120167,-0.284919,0.618597,-0.475856,-0.037987,-0.076373,-0.349231


In [3]:
def generate_batches(df, batch_size=32):
    start = 0
    end = len(df)

    while start < end - batch_size:
        batch_data = df.iloc[start: start+batch_size]
        batch = {
            'inputs': batch_data[df.columns[1:]].values.tolist(),
            'labels': batch_data[df.columns[0]].values.tolist()
        }

        yield batch

        start += batch_size

In [20]:
from time import time

start = time()

url = 'https://model-server-tihod4vvca-uc.a.run.app'

for i, batch in enumerate(generate_batches(test_df, batch_size=128)):
    if i >= 5:
        break

    data = { "instances": batch['inputs'] }
    json_data = json.dumps(data)

    response = requests.post(url, data=json_data, headers=headers)

    if response.status_code == 200:
        result = response.json()
        print("Inference Result:")
        print(result)
    else:
        # Error response
        print(f"Error: {response.status_code}, {response.text}")

print('Elapsed time: {:.4f}'.format(time() - start))

Inference Result:
[0.23191724717617035, 0.23191724717617035, 0.23191724717617035, 0.40438181161880493, 0.23191724717617035, 0.23191724717617035, 0.30337467789649963, 0.23191724717617035, 0.23191724717617035, 0.560187578201294, 0.23191724717617035, 0.23191724717617035, 0.23191724717617035, 0.23191724717617035, 0.23191724717617035, 0.23191724717617035, 0.30337467789649963, 0.23191724717617035, 0.23191724717617035, 0.23191724717617035, 0.23191724717617035, 0.23191724717617035, 0.23191724717617035, 0.23191724717617035, 0.30337467789649963, 0.23191724717617035, 0.560187578201294, 0.23191724717617035, 0.23191724717617035, 0.23191724717617035, 0.32006388902664185, 0.30337467789649963, 0.23191724717617035, 0.30337467789649963, 0.23191724717617035, 0.23191724717617035, 0.23191724717617035, 0.23191724717617035, 0.23191724717617035, 0.30337467789649963, 0.23191724717617035, 0.23191724717617035, 0.23191724717617035, 0.23191724717617035, 0.23191724717617035, 0.23191724717617035, 0.23191724717617035

In [21]:
import xgboost as xgb
import pickle
import numpy as np

with open('../../models/model/model.pkl', 'rb') as f:
    model = pickle.load(f)

In [24]:
start = time()

for i, batch in enumerate(generate_batches(test_df, batch_size=128)):
    if i >= 5:
        break

    data = np.array(batch['inputs'])

    inputs = xgb.DMatrix(data)

    result = model.predict(inputs)
    print('Inference Result:')
    print(list(result))
    

print('Elapsed time: {:.4f}'.format(time() - start))

Inference Result:
[0.23191725, 0.23191725, 0.23191725, 0.4043818, 0.23191725, 0.23191725, 0.30337468, 0.23191725, 0.23191725, 0.5601876, 0.23191725, 0.23191725, 0.23191725, 0.23191725, 0.23191725, 0.23191725, 0.30337468, 0.23191725, 0.23191725, 0.23191725, 0.23191725, 0.23191725, 0.23191725, 0.23191725, 0.30337468, 0.23191725, 0.5601876, 0.23191725, 0.23191725, 0.23191725, 0.3200639, 0.30337468, 0.23191725, 0.30337468, 0.23191725, 0.23191725, 0.23191725, 0.23191725, 0.23191725, 0.30337468, 0.23191725, 0.23191725, 0.23191725, 0.23191725, 0.23191725, 0.23191725, 0.23191725, 0.23191725, 0.23191725, 0.42774287, 0.5187841, 0.30337468, 0.23191725, 0.42774287, 0.23191725, 0.23191725, 0.23191725, 0.23191725, 0.23191725, 0.30337468, 0.23191725, 0.30337468, 0.23191725, 0.23191725, 0.23191725, 0.23191725, 0.23191725, 0.23191725, 0.23191725, 0.30337468, 0.23191725, 0.23191725, 0.23191725, 0.23191725, 0.30337468, 0.30337468, 0.23191725, 0.23191725, 0.3200639, 0.23191725, 0.23191725, 0.23191725, 0.2